## Modelo de datos de marcadores
#### Este modelo de datos se utiliza marcadores de tienda para los clientes.

Un cliente puede tener muchos marcadores y un marcador puede pertenecer a muchos clientes. Este modelo de datos representa una relación de muchos a muchos.

Los patrones de acceso que este modelo de datos facilita son:

* Una sola consulta de customerId ahora puede devolver datos de clientes, así como marcadores.

* Un índice ByEmail de consulta devuelve los datos del cliente por dirección de correo electrónico. Tenga en cuenta que este índice no recupera los marcadores.

* Un índice de consulta ByUrl obtiene datos de marcadores por URL. Tenga en cuenta que tenemos customerId como clave de ordenación para el índice porque la misma URL puede ser reservada por varios clientes.

* Un índice de consulta ByCustomerFolder obtiene marcadores por carpeta para cada cliente.

In [1]:
import pandas as pd
from spdynamodb import DynamoTable
import json
from decimal import Decimal
from time import sleep
import boto3

In [6]:
dt = DynamoTable()
try:
    dt.select_table('CustomerBookmark')
    print(dt)
except:
    dt.create_table(
        table_name='CustomerBookmark',
        partition_key='customerId',
        partition_key_type='N',
        sort_key="sk",
        sort_key_type="S"
)

Table created successfully!


In [7]:
dt.load_json("bookmarks.json")

#### [GSI] ByEmail

In [8]:
dt.create_global_secondary_index(
    att_name="email",
    att_type="S",
    i_name="ByEmail",
    proj_type=["sk", "fullName", "userPreferences"]
    i_rcu=1,
    i_wcu=1
)

In [9]:
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    sleep(30)
print("Global secondary index created.")

Global secondary index created.


#### [GSI] ByUrl

In [10]:
dt.create_global_secondary_index(
    att_name="url",
    att_type="S",
    sort_index="customerId",
    sort_type="N",
    i_name="ByUrl",
    proj_type=["sk", "createDate", "updateDate", "folder", "title", "description"],
    i_rcu=1,
    i_wcu=1
)

In [11]:
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    sleep(30)
print("Global secondary index created.")

Global secondary index created.


#### [GSI] ByCustomerFolder

In [12]:
dt.create_global_secondary_index(
    att_name="customerId",
    att_type="N",
    sort_index="folder",
    sort_type="S",
    i_name="ByCustomerFolder",
    proj_type=["sk", "createDate", "updateDate", "title", "description", "url"],
    i_rcu=1,
    i_wcu=1
)

In [13]:
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    sleep(30)
print("Global secondary index created.")

Global secondary index created.


In [14]:
dt.list_gsi

['ByEmail', 'ByUrl', 'ByCustomerFolder']